In [12]:
import warnings
warnings.filterwarnings('ignore')

from tools.docking.vina import vina_dock_multi
from tools.docking.preparation import define_vina_box
from example_notebooks.ui.file_widgets import PathSelector, DockButtons
import os, shutil

# Docking (vina) multiple ligands in a box defined by a reference ligand

### 1. Selecting some inputs:

---

**First we need to select some inputs, namely:**
- the protein we want to dock into (pdb file)
- the reference ligand to define a box (mol/sdf)
- the ligands we want to dock (sdf)

**To do this:**  
*(N.B. run the next cell to show the file dialogue)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

*repeat this until all files are selected*

In [17]:
f = PathSelector(start_dir=os.path.join(os.getcwd(), 'tools/docking/example_data'))
display(f.accord)
b = DockButtons(f)
b.render_buttons()

A Jupyter Widget

A Jupyter Widget

---

### 2. Check the inputs 
---

Run the next cell, and check you are happy with the inputs and docking directory

In [14]:
pdb_file = b.pdb_button.description
ligands_file = b.lig_button.description
ref_file = b.ref_button.description
conf_file = 'conf.txt'
docking_directory = os.path.join(os.getcwd(), 'dock/')
if os.path.isdir(docking_directory):
    shutil.rmtree(docking_directory)
os.mkdir(docking_directory)
os.chmod(docking_directory, 775)

print(f'inputs: \n\t - pdb: {pdb_file} \n\t - ligs: {ligands_file} \
      \n\t - ref: {ref_file} \n\ndocking directory: {docking_directory}')

inputs: 
	 - pdb: /code/tools/docking/example_data/md_middle_cluster.pdb 
	 - ligs: /code/tools/docking/example_data/3_mols.sdf       
	 - ref: /code/tools/docking/example_data/3_mols.sdf 

docking directory: /code/dock/


---

### 3. Define a box from the reference ligand
---

Run the next cell

In [15]:
define_vina_box(ligands_file, 'conf.txt')

---

### 4. Dock!
---

Run the next cell

In [16]:
vina_dock_multi(protein_pdb=pdb_file, ligands_sdf=ligands_file, 
                conf=conf_file, docking_dir=docking_directory, multiproc=True)

2019-08-08 21:39:19,550 - moleculekit.molecule - INFO - Removed 34 atoms. 5711 atoms remaining in the molecule.
2019-08-08 21:39:20,551 - propka - INFO - No pdbfile provided
2019-08-08 21:39:27,578 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.0.
2019-08-08 21:39:27,580 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+      2T A (pKa= 7.97)
2019-08-08 21:39:27,582 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+    181T B (pKa= 7.69)


'Docking complete! Your results have been saved to: \ndocked ligands: /code/dock/docked/ \nlog files: /code/dock/logs/ \nprepared input files: /code/dock/logs/'